# Computer-Robot Communication

ArdavanBidgoli <br />
CMU School of Architecture <br />
Robotic Plastering Project <br />
Tested with/for: <br />
OpenABB
The OpenABB project is adapted from: <br />
https://github.com/robotics/open_abb <br />
To add multiTasking and communication options, refer to this: <br />
https://forums.robotstudio.com/discussion/7839/how-to-add-options-to-already-built-in-system

In robot studio:   <br />
1- Add the SERVER.mod to the TRob task  <br />
2- Add a new task to the robot (RAPID> Controller> Configuration> Task) and add LOGGER.mod to it  <br />


# All importing and file setup

In [1]:
from abb import *
import copy
import sys
import os
import os.path
import json
import numpy as np

Run this twice!

In [3]:
import transformations 

For the camera functionalities

In [4]:
import urllib.request

### For the IP settings
Change tye IP address based on your run, if it is a simulation on the robot studio, use 127.0.0.1, if you are running the real code on the rovbot, use 128.2.109.20

In [5]:
isSimulation = True

In [6]:
if isSimulation:
    # the test IP for working with robot studio
    controllerIP = "127.0.0.1"
else:
    # This is the default IP for ABBs, change it based on your network
    controllerIP = "128.2.109.20"
    

serverPort = 5000
loggerPort = 5001

# Extra Functions

In [7]:
class RobotExtra(Robot):

    def __init__(self,
             ip = '128.2.109.20',
             port_motion = 5000,
             port_logger = 5001):
        print ("Making a robot: \nIP: \t\t%s \nMotion Port: \t%d \nMotion Port: \t%d"
                                                       %(ip, port_motion, port_logger))
        
        super().__init__(ip, port_motion, port_logger)
        

    def logger(self,message, fileName="log.txt",path = "./"):
        filePath = os.path.join(path, fileName)    
        with open(filePath, "wt") as myfile:
            myfile.write(message)
            myfile.close()
        print (message)   

# Helper functions

In [8]:
def goHome(robot, motion = 'cor'):
    # 'cor' for set_cartesian 
    # 'jnt' for set_joint
    if motion == 'cor':
        return robot.set_cartesian(homeCor)
    elif motion == 'jnt':
        return robot.set_joints(homeJnt)

def trigger_(delay = 2):
    print ("Taking a shot...")
    urllib.request.urlopen('http://10.5.5.9/gp/gpControl/command/shutter?p=1')
    time.sleep(delay)
    #print ("Image is captured")

def makePath(robot, dimension = 100 , size = 3):
    path = []
    pose = robot.get_cartesian()
    x, y = pose[0][0] , pose[0][1]
    print (x,y)
    for i in range (size):
        pose[0][0] = x
        x += dimension
        for j in range (size):
            pose[0][1] = y
            y += dimension
            print (pose)
            newPose = copy.deepcopy(pose)
            path.append(pose)
    return path

def euToQu(rot):
    # rot is a list of length 3, [x,y,z] rotations
    if type(rot) != list or len(rot) != 3: return None
    rot = np.deg2rad(rot)
    return transformations.quaternion_from_euler(rot[0],rot[1],rot[2],'sxyz')
    
def quToEu(rot):
    # rot is a list of length 4, [q1,q2,q3,q4] rotations
    if type(rot) != list or len(rot) != 4: return None
    result =  transformations.euler_from_quaternion(rot)
    return  np.rad2deg(result)

## Setting the Robot

In [9]:
# initiate an instance of robot class
myRobot = RobotExtra(ip = controllerIP)
print ("Robot is created!")
# Setting the environment
myRobot.set_units('millimeters','degrees')
myRobot.set_tool([[0,0,0], [1,0,0,0]])
myRobot.set_workobject([[0,0,0],[1,0,0,0]])

# set speed and zone 
myRobot.set_speed([1000,100,100,100])
myRobot.set_zone(zone_key='z0')

# define your home coordination and joint configuration
homeCor = [[1500,0,1500],[0,1,0,0]]
homeJnt = [0.0, 0.0, -0.0, 0.0, 30.0, -0.0]

Making a robot: 
IP: 		127.0.0.1 
Motion Port: 	5000 
Motion Port: 	5001
Robot is created!


## A test for surface scan
Make sure that you have an appropriate quaternion value for all the points <br />
The points in the grid are in this order: TopLeft, TopRight, BottomRight, BottomLeft

# Some extra functions as reminder
### Report the status
It is a simple report like this:  <br />
['1 VIRTUAL_USE', 'ROBOTWARE_6.03.2009', "IRB 6640-130/3.20'"]

In [10]:
myRobot.get_robotinfo()

['1 VIRTUAL_USE', 'ROBOTWARE_6.05.1049', "IRB 6640-185/2.75 LeanID'"]

Getting robot's current coordinations and joints

In [11]:
print (myRobot.get_cartesian())
print (myRobot.get_joints())

[[1764.74, 0.0, 2085.5], [0.5, 0.0, 0.866, 0.0]]
[0.0, 0.0, -0.0, 0.0, 30.0, -0.0]


### Motion Commands
at the end of set_cartesian it returns: b'1 1 ' <br />
at the end of set_joints it returns: b'2 1 '

In [13]:
# Define a coordination
c1 = [[1400,100,1600],[0,1,0,0]]
# move the robot to the designated coordination
myRobot.set_cartesian(c1)


b'1 1 '

In [14]:
# Define a joint setup 
j1 = [0.0, 0.0, -0.0, 0.0, 00.0, -0.0]
# move the robot to set joints to these values
myRobot.set_joints(j1)

b'2 1 '

In [15]:
myRobot.close()